In [6]:
import os
from glob import glob
import cv2 as cv
import numpy as np
from skimage.morphology import disk
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.saving import load_model
import pandas as pd
from scipy.spatial import ConvexHull
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression,LogisticRegressionCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import make_scorer,f1_score

# Preprocessing

In [ ]:
#Get all image paths from train set and create list of names for their destination after preprocessing

train_ims = sorted(glob(os.path.join('.\Train\Train','*.jpg')))
print(len(train_ims))
print(train_ims[:5])
filenametrain = []
for i in range(len(train_ims)):
    name = train_ims[i]
    name = name.replace('.\\Train\\Train\\','./Train/Modified/')
    filenametrain.append(name)
print(filenametrain[:5])

In [ ]:
#Do the same for the test set

test_ims = sorted(glob(os.path.join('.\Test\Test','*.jpg')))
print(len(test_ims))
print(test_ims[:5])
filenametest = []
for i in range(len(test_ims)):
    name = test_ims[i]
    name = name.replace('.\\Test\\Test\\','./Test/Modified/')
    filenametest.append(name)
print(filenametest[:5])

In [ ]:
#Define line kernel for the tophat and clahe operator for local histogram equalization to enhance image contrast
kernel = cv.getStructuringElement(1,(9,9))
clahe = cv.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))

In [ ]:
#Apply dull razor algorithm for hair removal then clahe on each image
for i in range(len(train_ims)):
    if i%100 ==0:
        print(i)
    im = cv.resize(cv.imread(train_ims[i],cv.IMREAD_GRAYSCALE),(512,384))
    blackhat = cv.morphologyEx(im, cv.MORPH_BLACKHAT, kernel)

    bhg= cv.GaussianBlur(blackhat,(3,3),cv.BORDER_DEFAULT)
    _,mask = cv.threshold(bhg,1,255,cv.THRESH_BINARY)

    dst = cv.inpaint(im,mask,6,cv.INPAINT_TELEA)

    mask = cv.dilate((dst<15).astype(np.uint8),disk(30))
    post = cv.inpaint(dst,mask,6,cv.INPAINT_TELEA)

    fim = clahe.apply(post)
    cv.imwrite(filenametrain[i],fim)

In [ ]:
for i in range(len(test_ims)):
    if i%100 ==0:
        print(i)
    im = cv.resize(cv.imread(test_ims[i],cv.IMREAD_GRAYSCALE),(512,384))
    blackhat = cv.morphologyEx(im, cv.MORPH_BLACKHAT, kernel)

    bhg= cv.GaussianBlur(blackhat,(3,3),cv.BORDER_DEFAULT)
    _,mask = cv.threshold(bhg,1,255,cv.THRESH_BINARY)

    dst = cv.inpaint(im,mask,6,cv.INPAINT_TELEA)

    mask = cv.dilate((dst<15).astype(np.uint8),disk(30))
    post = cv.inpaint(dst,mask,6,cv.INPAINT_TELEA)

    fim = clahe.apply(post)
    cv.imwrite(filenametest[i],fim)

# Define segmentation CNN

In [ ]:
#Get image paths for images that already have a segmentation as well as the path to this segmentation

seg_train = sorted(glob(os.path.join('.\Train\Train','*.png')))
print(len(seg_train))
print(seg_train[:5])
train_ims = []
for i in range(len(seg_train)):
    name = seg_train[i]
    name = name.replace('_seg.png','.jpg')
    name = name.replace('.\\Train\\Train\\','./Train/Modified/')
    train_ims.append(name)
print(train_ims[:5])



seg_test = sorted(glob(os.path.join('.\Test\Test','*.png')))
print(len(seg_test))
print(seg_test[:5])
test_ims = []
for i in range(len(seg_test)):
    name = seg_test[i]
    name = name.replace('_seg.png','.jpg')
    name = name.replace('.\\Test\\Test\\','./Test/Modified/')
    test_ims.append(name)
print(test_ims[:5])

In [ ]:
#Define CNN

in_layer = Input((384,512,1), name='input')
conv1 = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(in_layer)
conv2 = Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)(conv1)
conv3 = Conv2D(4, (3, 3), activation='relu', padding='same', strides=2)(conv2)
conv4 = Conv2DTranspose(4, kernel_size=3, strides=2, activation='relu', padding='same')(conv3)
conv5 = Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same')(conv4)
conv6 = Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same')(conv5)
out_layer = Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')(conv6)

autoencoder = keras.Model(in_layer,out_layer)
autoencoder.summary()

In [ ]:
# load image for training
autoencoder.compile(optimizer='adam', loss=MeanSquaredError())
train_ims = np.array([cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384))/255 for x in train_ims])
train_masks = np.array([cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384))/255 for x in seg_train])

test_ims = np.array([cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384))/255 for x in test_ims])
test_masks = np.array([cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384))/255 for x in seg_test])

print(train_ims.shape)
print(test_ims.shape)
print(train_masks.shape)
print(test_masks.shape)

In [ ]:
#Resahpe data
data = train_ims.reshape((1945,384,512,1))
masks = train_masks.reshape((1945,384,512,1))
test_data = test_ims.reshape((648,384,512,1))
test_masks = test_masks.reshape((648,384,512,1))

In [ ]:
#Fit model and save it
h = autoencoder.fit(data,masks,epochs=10,batch_size=32,verbose=1,shuffle=True,validation_data=(test_data,test_masks))
autoencoder.save('autoencoder_seg.keras')

# Finding best CNN output threshold

In [ ]:
#Get images and masks for all test images that already have masks
seg_test = sorted(glob(os.path.join('.\Test\Test','*.png')))
print(len(seg_test))
test_ims = []
for i in range(len(seg_test)):
    name = seg_test[i]
    name = name.replace('_seg.png','.jpg')
    name = name.replace('.\\Test\\Test\\','./Test/Modified/')
    test_ims.append(name)
print(test_ims[:5])

test_data = np.array([np.reshape(cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384)),(384,512,1)) for x in test_ims])
test_masks = 1/255*np.array([np.reshape(cv.resize(cv.imread(y,cv.IMREAD_GRAYSCALE),(512,384)),(384,512,1)) for y in seg_test])

In [ ]:
#Load model
autoencoder = load_model("autoencoder_seg.keras")
autoencoder.summary()

In [ ]:
#
s = np.zeros((648,255))
m = np.concatenate((np.zeros((384,64)),np.concatenate((np.ones((384,384)),np.zeros((384,64))),axis=1)),axis=1)

for i in range(0,648,2):
    print(i)
    z = test_data[i:i+2]/255
    p = autoencoder(z).numpy()
    for k in range(2):
        mask = test_masks[i+k]
        area = np.sum(mask)
        
        mi = np.min(p[k])
        ma = np.max(p[k])
        #Bring the image back between 0 and 255
        ad = 255/(ma-mi)*(p[k]-mi)
        
        for t in range(255):
            da = (ad>t).astype(np.uint8)

            #Verify is there are big objects on the sides
            s0 = np.sum(ad,axis=0)
            s1 = np.sum(s0[:128])
            s2 = np.sum(s0[128:256])
            s3 = np.sum(s0[256:384])
            if s1>s2 or s3>s2:
                da = (da.reshape(384,512)*m).astype(np.uint8)
            
            #Keep only the biggest object in the image
            da = cv.morphologyEx(da, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE, (60, 45)))
            da = np.pad(da,((1,1),(1,1)),'constant',constant_values=(0,0))
            ## Find largest contour in intermediate image
            cnts, _ = cv.findContours(da, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
            if len(cnts)!=0:
                cnt = max(cnts, key=cv.contourArea)
                # Output
                out = np.zeros(da.shape, np.uint8)
                cv.drawContours(out, [cnt], -1, 1, cv.FILLED)

                diff = np.abs(mask.reshape((384,512))-out[1:385,1:513])
                area_ratio = np.sum(diff)/area
                
                s[i+k,t] = area_ratio

err = np.sum(s,axis=0)
best_t = np.argmin(err)

# Segmenting images without masks

In [ ]:
ae = load_model("autoencoder_seg.keras")
ae.summary()
threshold = best_t

In [ ]:
# Keep only images who don't have a mask already
train_ims = sorted(glob(os.path.join('.\Train\Train','*.jpg')))
for i in range(len(train_ims)):
    train_ims[i] = train_ims[i].replace('.\\Train\\Train\\','./Train/Modified/')
print(len(train_ims))
print(train_ims[:5])

seg_ims = sorted(glob(os.path.join('.\Train\Train','*.png')))
print(len(seg_ims))

for i in range(len(seg_ims)):
    seg_ims[i] = seg_ims[i].replace('_seg.png','.jpg')
    seg_ims[i] = seg_ims[i].replace('.\\Train\\Train\\','./Train/Modified/')
print(seg_ims[:5])

undone_train = np.array([x for x in train_ims if x not in seg_ims])
print(undone_train[:5])


test_ims = sorted(glob(os.path.join('.\Test\Test','*.jpg')))
for i in range(len(test_ims)):
    test_ims[i] = test_ims[i].replace('.\\Test\\Test\\','./Test/Modified/')
print(len(test_ims))
print(test_ims[:5])

segtest_ims = sorted(glob(os.path.join('.\Test\Test','*.png')))
print(len(segtest_ims))

for i in range(len(segtest_ims)):
    segtest_ims[i] = segtest_ims[i].replace('_seg.png','.jpg')
    segtest_ims[i] = segtest_ims[i].replace('.\\Test\\Test\\','./Test/Modified/')
print(segtest_ims[:5])

undone_test = np.array([x for x in test_ims if x not in segtest_ims])
print(undone_test[:5])

In [ ]:
#Create names list for the future segmentation masks
filenametrain = []
for i in range(len(undone_train)):
    name = undone_train[i]
    name = name.replace('.\\Train\\Train\\','./Train/Modified/')
    name = name.replace('.jpg','_seg.png')
    filenametrain.append(name)
print(filenametrain[:5])

filenametest = []
for i in range(len(undone_test)):
    name = undone_test[i]
    name = name.replace('.\\Test\\Test\\','./Test/Modified/')
    name = name.replace('.jpg','_seg.png')
    filenametest.append(name)
print(filenametest[:5])

In [ ]:
#Compute and save masks for the train set
m = np.concatenate((np.zeros((384,64)),np.concatenate((np.ones((384,384)),np.zeros((384,64))),axis=1)),axis=1)
for i in range(0,len(undone_train),2):
    if i%100==0:
        print(i)
    z = np.array([1/255*np.reshape(cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384)),(384,512,1)) for x in undone_train[i:i+2]])
    p = ae(z).numpy()
    for k in range(2):
        mi = np.min(p[k])
        ma = np.max(p[k])
        im = 255/(ma-mi)*(p[k]-mi)
        thresh = (im>threshold).astype(np.uint8)

        s0 = np.sum(thresh,axis=0)
        s1 = np.sum(s0[:128])
        s2 = np.sum(s0[128:256])
        s3 = np.sum(s0[256:384])
        if s1>s2 or s3>s2:
            thresh = (thresh.reshape(384,512)*m).astype(np.uint8)
        
        thresh = cv.morphologyEx(thresh, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE, (60, 45)))
        thresh = np.pad(255*thresh,((1,1),(1,1)),'constant',constant_values=(0,0))
        # Find largest contour in intermediate image
        cnts, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
        if len(cnts)!=0:
            cnt = max(cnts, key=cv.contourArea)
            # Output
            out = np.zeros(thresh.shape, np.uint8)
            cv.drawContours(out, [cnt], -1, 255,cv.FILLED)
         
            cv.imwrite(filenametrain[i+k],out[1:385,1:513])

In [ ]:
#Do the same for the test set
for i in range(0,len(undone_test),2):
    if i%100==0:
        print(i)
    z = np.array([1/255*np.reshape(cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384)),(384,512,1)) for x in undone_test[i:i+2]])
    p = ae(z).numpy()
    for k in range(2):
        mi = np.min(p[k])
        ma = np.max(p[k])
        im = 255/(ma-mi)*(p[k]-mi)
        thresh = (im>threshold).astype(np.uint8)

        s0 = np.sum(thresh,axis=0)
        s1 = np.sum(s0[:128])
        s2 = np.sum(s0[128:256])
        s3 = np.sum(s0[256:384])
        if s1>s2 or s3>s2:
            thresh = (thresh.reshape(384,512)*m).astype(np.uint8)
        
        thresh = cv.morphologyEx(thresh, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE, (60, 45)))
        thresh = np.pad(255*thresh,((1,1),(1,1)),'constant',constant_values=(0,0))
        # Find largest contour in intermediate image
        cnts, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
        if len(cnts)!=0:
            cnt = max(cnts, key=cv.contourArea)
            # Output
            out = np.zeros(thresh.shape, np.uint8)
            cv.drawContours(out, [cnt], -1, 255,cv.FILLED)
         
            cv.imwrite(filenametest[i+k],out[1:385,1:513])

In [ ]:
#Add existing masks to the modified folders
seg_ims = sorted(glob(os.path.join('.\Train\Train','*.png')))
print(len(seg_ims))
print(seg_ims[:5])
train_dest = []
for i in range(len(seg_ims)):
    name = seg_ims[i]
    name = name.replace('.\\Train\\Train\\','./Train/Modified/')
    train_dest.append(name)
print(train_dest[:5])

for i in range(len(seg_ims)):
    im = cv.resize(cv.imread(seg_ims[i],cv.IMREAD_GRAYSCALE),(512,384))
    cv.imwrite(train_dest[i],im)



segtest_ims = sorted(glob(os.path.join('.\Test\Test','*.png')))
print(len(segtest_ims))
print(segtest_ims[:5])
test_dest = []
for i in range(len(segtest_ims)):
    name = segtest_ims[i]
    name = name.replace('.\\Test\\Test\\','./Test/Modified/')
    test_dest.append(name)
print(test_dest[:5])

for i in range(len(segtest_ims)):
    im = cv.resize(cv.imread(segtest_ims[i],cv.IMREAD_GRAYSCALE),(512,384))
    cv.imwrite(test_dest[i],im)

In [ ]:
#Set black masks for the few images that don't have any
segtest_ims = sorted(glob(os.path.join('.\Test\Modified','*.png')))
print(len(segtest_ims))
print(segtest_ims[:5])
test_ims = sorted(glob(os.path.join('.\Test\Modified','*.jpg')))
print(len(test_ims))
print(test_ims[:5])

leftout_test=[]
segtest_names = []
for i in range(len(segtest_ims)):
    name = segtest_ims[i]
    name = name.replace('.\\Test\\Modified\\','')
    name = name.replace('_seg.png','')
    segtest_names.append(name)
test_names = []
for i in range(len(test_ims)):
    name = test_ims[i]
    name = name.replace('.\\Test\\Modified\\','')
    name = name.replace('.jpg','')
    test_names.append(name)
print(segtest_names[:5])
print(test_names[:5])
leftout_test = [x for x in test_names if x not in segtest_names]
print(len(leftout_test))
print(leftout_test[:5])
leftout_seg = []
for i in range(len(leftout_test)):
    name = leftout_test[i]
    name = './Test/Modified/'+name+'_seg.png'
    leftout_seg.append(name)
print(leftout_seg[:5])
for i in range(len(leftout_seg)):
    im = np.zeros((384,512))
    cv.imwrite(leftout_seg[i],im)




segtrain_ims = sorted(glob(os.path.join('.\Train\Modified','*.png')))
print(len(segtrain_ims))
print(segtrain_ims[:5])
train_ims = sorted(glob(os.path.join('.\Train\Modified','*.jpg')))
print(len(train_ims))
print(train_ims[:5])

leftout_train=[]
segtrain_names = []
for i in range(len(segtrain_ims)):
    name = segtrain_ims[i]
    name = name.replace('.\\Train\\Modified\\','')
    name = name.replace('_seg.png','')
    segtrain_names.append(name)
train_names = []
for i in range(len(train_ims)):
    name = train_ims[i]
    name = name.replace('.\\Train\\Modified\\','')
    name = name.replace('.jpg','')
    train_names.append(name)
print(segtrain_names[:5])
print(train_names[:5])
leftout_train = [x for x in train_names if x not in segtrain_names]
print(len(leftout_train))
print(leftout_train[:5])
leftout_seg = []
for i in range(len(leftout_train)):
    name = leftout_train[i]
    name = './Train/Modified/'+name+'_seg.png'
    leftout_seg.append(name)
print(leftout_seg[:5])
for i in range(len(leftout_seg)):
    im = np.zeros((384,512))
    cv.imwrite(leftout_seg[i],im)

# Filling Missing metadata values

In [ ]:
train_data = pd.read_csv('./metadataTrain.csv')
train_data.head()

In [ ]:
#Taking care of missing sex values
missing_sex = train_data[train_data['SEX'].isna()]
print(missing_sex.count())
print(missing_sex.head())

sex = train_data[train_data['SEX'].isin(['male','female'])]
print(sex.count())
print(sex.head())

fem = sex[sex['SEX']=='female']
print(fem.count())
print(fem.head())

mal = sex[sex['SEX']=='male']
print(mal.count())
print(mal.head())

male_prob = mal['ID'].count()/sex['ID'].count()
print(male_prob)

def f(x):
    if x in ['male','female']:
        return x
    else:
        p = np.random.random()
        if p>male_prob:
            return 'female'
        else:
            return 'male'
    
train_data['SEX'] = train_data['SEX'].map(f)
train_data.head()

In [ ]:
#Taking care of missing age values
age = train_data[train_data['AGE'].notna()]
age.count()

agenp = age['AGE'].to_numpy()
print(agenp)
min_age = np.min(agenp)
max_age = np.max(agenp)
print(min_age,max_age)

def g(x):
    if np.isnan(x):
        return np.random.randint(min_age,max_age+1)
    else:
        return x
    
train_data['AGE'] = train_data['AGE'].map(g)
train_data.head()

In [ ]:
#Taking care of missing position
print(train_data['POSITION'].dropna().count())
poss = np.unique(train_data['POSITION'].dropna().to_numpy())
print(poss)

good_pos = train_data['POSITION'].dropna()

ant = good_pos[good_pos==poss[0]].count()
head = good_pos[good_pos==poss[1]].count()
lat = good_pos[good_pos==poss[2]].count()
low = good_pos[good_pos==poss[3]].count()
ora = good_pos[good_pos==poss[4]].count()
pal = good_pos[good_pos==poss[5]].count()
pos = good_pos[good_pos==poss[6]].count()
upp = good_pos[good_pos==poss[7]].count()
tot = ant+head+lat+low+ora+pal+pos+upp
print(ant,head,lat,low,ora,pal,pos,upp)
print(tot)
#Compute probabilities of each position
cnts = [ant,head,lat,low,ora,pal,pos,upp]
probs = cnts/tot
ints = np.cumsum(probs)
print(cnts)
print(probs)
print(ints)

def h(x):
    if x in poss:
        return x
    else:
        p = np.random.random()
        if p<ints[0]:
            return poss[0]
        elif p<ints[1]:
            return poss[1]
        elif p<ints[2]:
            return poss[2]
        elif p<ints[3]:
            return poss[3]
        elif p<ints[4]:
            return poss[4]
        elif p<ints[5]:
            return poss[5]
        elif p<ints[6]:
            return poss[6]
        else:
            return poss[7]       
    
train_data['POSITION'] = train_data['POSITION'].map(h)
train_data.head()

In [ ]:
print(train_data.notna().count())

In [ ]:
train_data.to_csv('./metadataTrainFilled.csv',index=False)

In [ ]:
#Now do the same for test metadata
test_data = pd.read_csv('./metadataTest.csv')
missing_sex = test_data[test_data['SEX'].isna()]
sex = test_data[test_data['SEX'].isin(['male','female'])]
mal = sex[sex['SEX']=='male']
male_prob = mal['ID'].count()/sex['ID'].count()

def f(x):
    if x in ['male','female']:
        return x
    else:
        p = np.random.random()
        if p>male_prob:
            return 'female'
        else:
            return 'male'
    
test_data['SEX'] = test_data['SEX'].map(f)



age = test_data[test_data['AGE'].notna()]
agenp = age['AGE'].to_numpy()
min_age = np.min(agenp)
max_age = np.max(agenp)

def g(x):
    if np.isnan(x):
        return np.random.randint(min_age,max_age+1)
    else:
        return x
    
test_data['AGE'] = test_data['AGE'].map(g)



poss = np.unique(test_data['POSITION'].dropna().to_numpy())
good_pos = test_data['POSITION'].dropna()

ant = good_pos[good_pos==poss[0]].count()
head = good_pos[good_pos==poss[1]].count()
lat = good_pos[good_pos==poss[2]].count()
low = good_pos[good_pos==poss[3]].count()
ora = good_pos[good_pos==poss[4]].count()
pal = good_pos[good_pos==poss[5]].count()
pos = good_pos[good_pos==poss[6]].count()
upp = good_pos[good_pos==poss[7]].count()
tot = ant+head+lat+low+ora+pal+pos+upp

cnts = [ant,head,lat,low,ora,pal,pos,upp]
probs = cnts/tot
ints = np.cumsum(probs)

def h(x):
    if x in poss:
        return x
    else:
        p = np.random.random()
        if p<ints[0]:
            return poss[0]
        elif p<ints[1]:
            return poss[1]
        elif p<ints[2]:
            return poss[2]
        elif p<ints[3]:
            return poss[3]
        elif p<ints[4]:
            return poss[4]
        elif p<ints[5]:
            return poss[5]
        elif p<ints[6]:
            return poss[6]
        else:
            return poss[7]       
    
test_data['POSITION'] = test_data['POSITION'].map(h)

print(test_data.notna().count())


test_data.to_csv('./metadataTestFilled.csv',index=False)

# Compute features

In [ ]:
classes = pd.DataFrame(index=[1,2,3,4,5,6,7,8], data ={'name':['Melanoma','Melanocytic nevus',
'Basal cell carcinoma',
 'Actinic keratosis',
 'Benign keratosis',
 'Dermatofibroma', 
 'Vascular lesion',
 'Squamous cell carcinoma']},)
classes

In [ ]:
#Loading metadata
data = pd.read_csv('metadataTrainFilled.csv')
print('Number of training images:',data['ID'].count())
print('Missing number of sexes in training set:',data['SEX'].isna().sum())
print('Missing number of ages in training set:',data['AGE'].isna().sum())
print('Missing number of locations in training set:',data['POSITION'].isna().sum())
data.head()

test = pd.read_csv('metadataTestFilled.csv')
print('Number of testing images:',test['ID'].count())
print('Missing number of sexes in testing set:',test['SEX'].isna().sum())
print('Missing number of ages in testing set:',test['AGE'].isna().sum())
print('Missing number of locations in testing set:',test['POSITION'].isna().sum())
test.head()

In [ ]:
#Transform exisiting metadata into numbers between 0 and 1
def f(x):
    if x == 'male':
        return 0
    else:
        return 1
    
data['SEX'] = data['SEX'].map(f)
test['SEX'] = test['SEX'].map(f)

agenp = data['AGE'].to_numpy()
#min_age = np.min(agenp) # is actually 0 so not I can just divide by the maximum
max_age = np.max(agenp)

def g(x):
    return x/max_age
    
data['AGE'] = data['AGE'].map(g)
test['AGE'] = test['AGE'].map(g)

poss = np.unique(data['POSITION'].to_numpy())
def h(x):
    if x == poss[0]:
        return 0
    elif x == poss[2]:
        return 0.14
    elif x == poss[6]:
        return 0.29
    elif x == poss[1]:
        return 0.43
    elif x == poss[4]:
        return 0.57
    elif x == poss[7]:
        return 0.71
    elif x == poss[3]:
        return 0.86
    elif x == poss[5]:
        return 1
    
data['POSITION'] = data['POSITION'].map(h)
test['POSITION'] = test['POSITION'].map(h)

In [ ]:
def seg_features(mask):
    #Area
    area = np.sum(mask)

    if area>2:
        #Perimeter
        cont_test = np.zeros(mask.shape)
        Contour, Hierarchy = cv.findContours(mask.astype(np.uint8), cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
        cv.drawContours(cont_test, Contour, len(Hierarchy[0])-1, 1, 1)
        perim = np.sum(cont_test)
        #Convex hull
        coo = np.where(mask>0)
        x = coo[0]
        y = coo[1]
        xm,ym = np.mean(x),np.mean(y)
        coord = np.array([[a,b] for a,b in zip(x,y)])
        try:
            hull = ConvexHull(coord)
            conv_perim = hull.area
            conv_area = hull.volume
            #Compactness, Roundness, Convexity, Solidity
            compac = 4*np.pi*area/(perim**2)
            roundn = 4*np.pi*area/(conv_perim**2)
            convex = conv_perim/perim
            solid = area/conv_area
        except:
            conv_perim = 0
            conv_area = 0
            compac = 0
            roundn = 0
            convex = 0
            solid = 0
        #Center of bb and aspect ratio
        suml = np.sum(cont_test,axis=0)
        sumc = np.sum(cont_test,axis=1)
        indl = np.where(suml>0)[0]
        indc = np.where(sumc>0)[0]
        x1,x2,y1,y2 = indl[0],indl[-1],indc[0],indc[-1]
        xc,yc = (x1+x2)/2,(y1+y2)/2
        height,width = np.min((x2-x1,y2-y1)),np.max((x2-x1,y2-y1))
        aspectratio = height/width
        dist = np.sqrt((xc-xm)**2+(yc-ym)**2)
    else:
        perim = 0
        aspectratio = 0
        dist = 0
        compac = 0
        roundn = 0
        convex = 0
        solid = 0

    return area,perim,aspectratio,dist,compac,roundn,convex,solid

def channel_features(im,mask):

    b,g,r = cv.split(im)
    hls = cv.cvtColor(im,cv.COLOR_BGR2HLS)
    h,l,_ = cv.split(hls)
    #Min, Max, Average and Variance of L
    mil = np.min(l)
    mal = np.max(l)
    avl = np.mean(l)
    val = np.var(l)
    #Min, Max, Average and Variance of H
    mih = np.min(h)
    mah = np.max(h)
    avh = np.mean(h)
    vah = np.var(h)
    #Values of B,G,R,H and L inside the lesion and outside of it
    bc = np.mean(b*mask)
    be = np.mean(b*(1-mask))
    gc = np.mean(g*mask)
    ge = np.mean(g*(1-mask))
    rc = np.mean(r*mask)
    re = np.mean(r*(1-mask))
    hc = np.mean(h*mask)
    he = np.mean(h*(1-mask))
    lc = np.mean(l*mask)
    le = np.mean(l*(1-mask))
    #Difference between the above values
    diffb = abs(bc-be)
    diffg = abs(gc-ge)
    diffr = abs(rc-re)
    diffh = abs(hc-he)
    diffl = abs(lc-le)


    return mil,mal,avl,val,mih,mah,avh,vah,diffb,diffg,diffr,diffh,diffl

In [ ]:
#Compute features for the train set and save them in a csv file
names = data['ID'].to_numpy()
minl,maxl,avgl,varl,minh,maxh,avgh,varh,diffb,diffg,diffr,diffh,diffl = [],[],[],[],[],[],[],[],[],[],[],[],[]
area,perimeter,aspectratio,distance,compactness,roundness,convexity,solidity = [],[],[],[],[],[],[],[]

for i in range(len(names)):
    if i%100 == 0:
        print(i)

    path = './Train/Train/'+names[i]+'.jpg'
    im = cv.resize(cv.imread(path),(512,384))
    maskpath = './Train/Modified/'+names[i]+'_seg.png'
    mask = 1/255*cv.imread(maskpath,cv.IMREAD_GRAYSCALE)

    mil,mal,avl,val,mih,mah,avh,vah,dib,dig,di_r,dih,dil = channel_features(im,mask)
    ar,perim,asp,dist,compact,roundn,convex,solid = seg_features(mask)
    
    minl.append(mil)
    maxl.append(mal)
    avgl.append(avl)
    varl.append(val)

    minh.append(mih)
    maxh.append(mah)
    avgh.append(avh)
    varh.append(vah)

    diffb.append(dib)
    diffg.append(dig)
    diffr.append(di_r)
    diffh.append(dih)
    diffl.append(dil)

    area.append(ar)
    perimeter.append(perim)
    aspectratio.append(asp)
    distance.append(dist)
    compactness.append(compact)
    roundness.append(roundn)
    convexity.append(convex)
    solidity.append(solid)

#Put values between 0 and 1
minl = (minl-np.min(minl))/(np.max(minl)-np.min(minl))
maxl = (maxl-np.min(maxl))/(np.max(maxl)-np.min(maxl))
avgl = (avgl-np.min(avgl))/(np.max(avgl)-np.min(avgl))
varl = (varl-np.min(varl))/(np.max(varl)-np.min(varl))

minh = (minh-np.min(minh))/(np.max(minh)-np.min(minh))
maxh = (maxh-np.min(maxh))/(np.max(maxh)-np.min(maxh))
avgh = (avgh-np.min(avgh))/(np.max(avgh)-np.min(avgh))
varh = (varh-np.min(varh))/(np.max(varh)-np.min(varh))

diffb = (diffb-np.min(diffb))/(np.max(diffb)-np.min(diffb))
diffg = (diffg-np.min(diffg))/(np.max(diffg)-np.min(diffg))
diffr = (diffr-np.min(diffr))/(np.max(diffr)-np.min(diffr))
diffh = (diffh-np.min(diffh))/(np.max(diffh)-np.min(diffh))
diffl = (diffl-np.min(diffl))/(np.max(diffl)-np.min(diffl))

area = (area-np.min(area))/(np.max(area)-np.min(area))
perimeter = (perimeter-np.min(perimeter))/(np.max(perimeter)-np.min(perimeter))
distance = (distance-np.min(distance))/(np.max(distance)-np.min(distance))


id = data['ID'].to_numpy()
clas = data['CLASS'].to_numpy()
sex = data['SEX'].to_numpy()
age = data['AGE'].to_numpy()
pos = data['POSITION'].to_numpy()
#Had to put a maximum of 1 for thee values that shouldn't exceed it by definition
compactness = np.minimum(compactness,1)
roundness = np.minimum(roundness,1)
convexity = np.minimum(convexity,1.3)/1.3
solidity = np.minimum(solidity,1)

dic = {'ID':id,'CLASS':clas,'SEX':sex,'AGE':age,'POSITION':pos,'MinL':minl,'MaxL':maxl,'AvgL':avgl,'VarL':varl,'MinH':minh,'MaxH':maxh,'AvgH':avgh,'VarH':varh,'DiffB':diffb,
        'DiffG':diffg,'DiffR':diffr,'DiffH':diffh,'DiffL':diffl,'Area':area,'Perimeter':perimeter,'Aspect Ratio':aspectratio,'Distance':distance,'Compactness':compactness,
        'Roundness':roundness,'Convexity':convexity,'Solidity':solidity}
print(len(dic.keys()))

df = pd.DataFrame(dic)
df = df.fillna(0)
print(df.head())

df.to_csv('./MetadataTrainModified.csv',index=False)

In [ ]:
#Compute the features for the test set
names = test['ID'].to_numpy()
minl,maxl,avgl,varl,minh,maxh,avgh,varh,diffb,diffg,diffr,diffh,diffl = [],[],[],[],[],[],[],[],[],[],[],[],[]
area,perimeter,aspectratio,distance,compactness,roundness,convexity,solidity = [],[],[],[],[],[],[],[]

for i in range(len(names)):
    if i%100 == 0:
        print(i)

    path = './Test/Test/'+names[i]+'.jpg'
    im = cv.resize(cv.imread(path),(512,384))
    maskpath = './Test/Modified/'+names[i]+'_seg.png'
    mask = 1/255*cv.imread(maskpath,cv.IMREAD_GRAYSCALE)

    mil,mal,avl,val,mih,mah,avh,vah,dib,dig,di_r,dih,dil = channel_features(im,mask)
    ar,perim,asp,dist,compact,roundn,convex,solid = seg_features(mask)
    
    minl.append(mil)
    maxl.append(mal)
    avgl.append(avl)
    varl.append(val)

    minh.append(mih)
    maxh.append(mah)
    avgh.append(avh)
    varh.append(vah)

    diffb.append(dib)
    diffg.append(dig)
    diffr.append(di_r)
    diffh.append(dih)
    diffl.append(dil)

    area.append(ar)
    perimeter.append(perim)
    aspectratio.append(asp)
    distance.append(dist)
    compactness.append(compact)
    roundness.append(roundn)
    convexity.append(convex)
    solidity.append(solid)

#Put values between 0 and 1
minl = (minl-np.min(minl))/(np.max(minl)-np.min(minl))
maxl = (maxl-np.min(maxl))/(np.max(maxl)-np.min(maxl))
avgl = (avgl-np.min(avgl))/(np.max(avgl)-np.min(avgl))
varl = (varl-np.min(varl))/(np.max(varl)-np.min(varl))

minh = (minh-np.min(minh))/(np.max(minh)-np.min(minh))
maxh = (maxh-np.min(maxh))/(np.max(maxh)-np.min(maxh))
avgh = (avgh-np.min(avgh))/(np.max(avgh)-np.min(avgh))
varh = (varh-np.min(varh))/(np.max(varh)-np.min(varh))

diffb = (diffb-np.min(diffb))/(np.max(diffb)-np.min(diffb))
diffg = (diffg-np.min(diffg))/(np.max(diffg)-np.min(diffg))
diffr = (diffr-np.min(diffr))/(np.max(diffr)-np.min(diffr))
diffh = (diffh-np.min(diffh))/(np.max(diffh)-np.min(diffh))
diffl = (diffl-np.min(diffl))/(np.max(diffl)-np.min(diffl))

area = (area-np.min(area))/(np.max(area)-np.min(area))
perimeter = (perimeter-np.min(perimeter))/(np.max(perimeter)-np.min(perimeter))
distance = (distance-np.min(distance))/(np.max(distance)-np.min(distance))

id = test['ID'].to_numpy()
sex = test['SEX'].to_numpy()
age = test['AGE'].to_numpy()
pos = test['POSITION'].to_numpy()
#Had to put a maximum of 1 for thee values that shouldn't exceed it by definition
compactness = np.minimum(compactness,1)
roundness = np.minimum(roundness,1)
convexity = np.minimum(convexity,1.3)/1.3
solidity = np.minimum(solidity,1)

dic = {'ID':id,'SEX':sex,'AGE':age,'POSITION':pos,'MinL':minl,'MaxL':maxl,'AvgL':avgl,'VarL':varl,'MinH':minh,'MaxH':maxh,'AvgH':avgh,'VarH':varh,'DiffB':diffb,
        'DiffG':diffg,'DiffR':diffr,'DiffH':diffh,'DiffL':diffl,'Area':area,'Perimeter':perimeter,'Aspect Ratio':aspectratio,'Distance':distance,'Compactness':compactness,
        'Roundness':roundness,'Convexity':convexity,'Solidity':solidity}

df = pd.DataFrame(dic)
df.fillna(0)
print(df.head())


df.to_csv('./MetadataTestModified.csv',index=False)

# Classification

In [ ]:
# Load new metadata
train = pd.read_csv('MetadataTrainModified.csv')
print(train.head())
test = pd.read_csv('MetadataTestModified.csv')
print(test.head())

#Convert to numpy array
t = train.to_numpy()
print(t.shape)
t2 = test.to_numpy()
print(t2.shape)

#Separate between classes and metadata
train_labels = np.array(t[:,1]).astype(int)
#print(train_labels)
train_data = t[:,2:]
#print(train_data)
test_data = t2[:,1:]
#print(test_data)
ids = t2[:,0]

#Apply onehotencoder to improve results on certain classifiers
train_lab = train_labels.reshape(-1,1)
ohe = OneHotEncoder()
ohe.fit(train_lab)
labels = ohe.transform(train_lab).toarray()
#print(labels.shape)
#print(labels)

## Part 1 : Linear model

In [ ]:
reg = LinearRegression()
reg.fit(train_data,labels)
print(reg.score(train_data,labels))

test_labels = reg.predict(test_data)
test_labels = np.argmax(test_labels,axis=1)+1
print(test_labels)

dict = {'ID':ids,'CLASS':test_labels}
df.pd.DataFrame(dict)
df.to_csv('./LinearClassif.csv',index=False)

## Part 2: Logistic Regression

In [ ]:
logitcv= LogisticRegressionCV(Cs=[0.5,0.75,1],cv=10,penalty='l2',class_weight='balanced',solver='saga',max_iter=1000 ,multi_class='multinomial')
logitcv.fit(train_data,train_labels)
print(logitcv.score(train_data,train_labels))
print(logitcv.C_)

In [ ]:

logit = LogisticRegression(penalty='l2',C=0.5,class_weight='balanced',solver='saga',max_iter=10000 ,multi_class='multinomial')
logit.fit(train_data,train_labels)
print(logit.score(train_data,train_labels))

test_labels = logit.predict(test_data)
print(test_labels)

dict = {'ID':ids,'CLASS':test_labels}
df.pd.DataFrame(dict)
df.to_csv('./Logit.csv',index=False)

## Part 3: Trees

### I - Decision tree

In [ ]:
tree = DecisionTreeClassifier(criterion='entropy',random_state=23)
tree.fit(train_data,labels)
print(tree.score(train_data,labels))
test_labels = tree.predict(test_data)
test_labels = np.argmax(test_labels,axis=1)+1

dict = {'ID':ids,'CLASS':test_labels}
df.pd.DataFrame(dict)
df.to_csv('./DecisionTree.csv',index=False)

### II - Random Forest

In [ ]:
RF =RandomForestClassifier(criterion='entropy',random_state=23)
p_grid_RF = {'n_estimators': [10,15,20,25,30], 'min_samples_leaf': [2,3,4,5,6], 'max_features': ['sqrt','log2']}   

grid_RF = GridSearchCV(estimator=RF, param_grid=p_grid_RF, scoring='balanced_accuracy', cv=5,verbose=2)
grid_RF.fit(train_data,labels)
print(grid_RF.best_score_)
print(grid_RF.best_params_)

In [ ]:
trees = RandomForestClassifier(n_estimators=100,criterion='entropy',max_features='sqrt',random_state=23)
trees.fit(train_data,labels)
print(trees.score(train_data,labels))
test_labels = trees.predict(test_data)
test_labels = np.argmax(test_labels,axis=1)+1

dict = {'ID':ids,'CLASS':test_labels}
df.pd.DataFrame(dict)
df.to_csv('./RandomForest.csv',index=False)

## Part 4: SVM

In [ ]:
p_grid={'C':[10,15,20],'gamma':[0.1,0.2,0.5,1]}
grid_svm = GridSearchCV(SVC(),p_grid,scoring='balanced_accuracy',verbose=3)
grid_svm.fit(train_data,train_labels)
print(grid_svm.best_params_)

In [ ]:
svm = SVC(verbose=True,C=20,gamma=1,class_weight='balanced')
svm.fit(train_data,train_labels)
print(svm.score(train_data,train_labels))
test_labels = svm.predict(test_data)

dict = {'ID':ids,'CLASS':test_labels}
df.pd.DataFrame(dict)
df.to_csv('./SVM.csv',index=False)